In [ ]:

# This is the main torch package
import torch 
#Computer vision specific package              
import torchvision
#There are a bunch of standard datasets in torchvision. 
import torchvision.datasets as datasets


# Librosa for processing datasets

import librosa as lb

# Import Google Drive Access
from google.colab import files

# OS and Pickle Imports for saving CQT representations and trained network in drive
import os
import pickle


import numpy as np


## Data Helper Functions



In [ ]:
# Google Drive mounting for data access
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Convert .mp3 files from datasets into Constant-Q transforms
def song_to_cqt(song_path):
  song, sampling_rate = lb.load(song_path)
  C= lb.cqt(song,sampling_rate)
  desire_spect_len = 2580
  # if spectral respresentation too long, crop it, otherwise, zero-pad
  if C.shape[1] >= desire_spect_len:
    C = C[:,0:desire_spect_len]
  else:
    C = np.pad(C,((0,0),(0,desire_spect_len-C.shape[1])), 'constant')
  return C

# Create a Matrix for every song to keep track of which song has which CQT representation
# {Key: 'Song', Value: CQT representation}
def create_feature_matrix(song_folder_path):
  reference_feature_matrix = {}
  cover_feature_matrix = {}
  i = 0
  for songname in os.listdir(song_folder_path):
    if songname!='.DS_Store':
      song_folder = song_folder_path + '/' + songname
      for song in os.listdir(song_folder):
        song_path = song_folder + '/' + song
        if song.find('Cover') != -1:
          cover_feature_matrix[song] = song_to_cqt(song_path)
        else: 
          ref_song = song[0:song.find('Original')]
          reference_feature_matrix[ref_song] = song_to_cqt(song_path)
    i = i + 1
    print(i)
  return cover_feature_matrix, reference_feature_matrix

#Save Feature Matrix to drive so songs only need to be converted to CQT once
def save_feature_matrix(song_folder_path, save_path):
  cover_songs, reference_songs = create_feature_matrix(song_folder_path)
  print("")
  print('Saving Feature Matrix')
  r_outfile = open(save_path + 'references','wb')
  pickle.dump(reference_songs, r_outfile)
  r_outfile.close()
  c_outfile = open(save_path + 'covers','wb')
  pickle.dump(cover_songs, c_outfile)
  c_outfile.close()


      

# Load Training Data

In [ ]:
### Run this Cell only once ###
# Saves CQT of covers and reference songs to Google Drive 
train_set_path = '/content/drive/MyDrive/Deep Learning Final Project/secondhandsongs'
train_store_path = '/content/drive/MyDrive/Deep Learning Final Project/train_data/'

save_feature_matrix(train_set_path, train_store_path)


print('Load Completed')


In [ ]:
### Run Every Time Notebook Resets ###

# Load Cover Song and Refernce Song CQT representation from Google Drive

covers_path = '/content/drive/MyDrive/Deep Learning Final Project/train_data/covers'
references_path = '/content/drive/MyDrive/Deep Learning Final Project/train_data/references'

infile = open(covers_path,'rb')
cover_songs = pickle.load(infile)
infile.close()

infile = open(references_path,'rb')
reference_songs = pickle.load(infile)
infile.close()


# Convert Feature Matrix to training data (tuples of song pairs) and labels (0 if cover, 1 if not cover)

train_data = []
train_labels = np.array([])
for reference_song in reference_songs:
  for cover_song in cover_songs:
      label = int(cover_song.find(reference_song) == -1)
      train_labels = np.append(train_labels,label)
      song_tuple = (torch.from_numpy(cover_songs[cover_song]), torch.from_numpy(reference_songs[reference_song]))
      train_data.append(song_tuple)


# Custom PyTorch Dataset for Songs

In [ ]:
from torch.utils.data import DataLoader,Dataset
import random

# Custom dataset for cover song identification
# (song0,song1) are from the reference set and cover set respectively
# Label is 0 if song1 is a cover of song 0
class CoverSongDataset(Dataset):
    
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels
    
    def __getitem__(self,index):
        #we need to make sure approx 50% of song pairs are cover/reference pairs
        should_be_cover = random.randint(0,1) 

        if should_be_cover:
          while True:
            rand_ind = random.randint(0,len(self.labels)-1)
            if self.labels[rand_ind] == 0:
              song0,song1 = self.data[rand_ind]
              lab = self.labels[rand_ind]
              break
        else: 
          while True:
            rand_ind = random.randint(0,len(self.labels)-1)
            if self.labels[rand_ind] == 1:
              song0,song1 = self.data[rand_ind]
              lab = self.labels[rand_ind]
              break

        return song0, song1 , torch.from_numpy(np.array([lab],dtype=np.float32))
    
    def __len__(self):
        return len(self.labels)



# Siamese Network Architecture


In [ ]:
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Siamese Convolutional Network 
# Input is two songs, output is a predicted label of that song
class Siamese(nn.Module):
    def __init__(self):
      super(Siamese, self).__init__()
      self.main = nn.Sequential(
            nn.Conv2d(1, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d((2,4)),
            nn.BatchNorm2d(128),
            nn.Dropout(.3),
            nn.Conv2d(128, 96, 3),
            nn.ReLU(),
            nn.MaxPool2d((3,5)),
            nn.BatchNorm2d(96),
            nn.Dropout(.5),
            nn.Conv2d(96, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d((3,8)),
            nn.BatchNorm2d(64),
            nn.Dropout(.5),
            nn.Conv2d(64, 32, 3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(.5),
        )
      self.sigmoid = nn.Sigmoid()
      self.fc1 = nn.Linear(32*13,416)
      self.fcOut = nn.Linear(416,1)


    def forward(self, x1, x2):
        x1 = self.main(x1)
        x1= x1.view(-1,32*13)
        x1 = self.sigmoid(self.fc1(x1))

        x2 = self.main(x2)
        x2= x2.view(-1,32*13)
        x2 = self.sigmoid(self.fc1(x2))
        x = torch.abs(x1 - x2)
        x= self.fcOut(x)
        return x
        




# Training the Siamese Network

In [ ]:
# Define training parameters

siamese_dataset = CoverSongDataset(train_data, train_labels)

net = Siamese().cuda()
criterion = F.mse_loss
optimizer = optim.Adam(net.parameters(),lr = 0.001 )

num_epochs=5

train_dataloader = DataLoader(siamese_dataset,
                        batch_size=16)

In [ ]:
import matplotlib.pyplot as plt

epoch_avg_loss = []
epoch_med_loss = []
x = []
loss_mat = []

# Train over 5 epoch with a batch size of 16

for epoch in range(0,num_epochs):
  print(epoch)
  net.train()
  for i, data in enumerate(train_dataloader, 0):
        song0,song1,label = data
        song0, song1 , label = song0.cuda(), song1.cuda() , label.cuda()
        song0 = song0.unsqueeze(1)
        song1 = song1.unsqueeze(1)
        output = net(torch.abs(song0).float(),torch.abs(song1).float())
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        loss_mat.append(loss.item())

        # Calculate loss every 500 steps for plot
        
        if i%500==0:
          avg_loss = np.mean(loss_mat)
          med_loss = np.median(loss_mat)
          epoch_avg_loss.append(avg_loss)
          epoch_med_loss.append(med_loss)
          curr_step = 3816 * epoch + i
          x.append(curr_step)
          print(str(curr_step))
          print(avg_loss)
          print(med_loss)
          loss_mat = []

  print("")



plt.plot(x,epoch_avg_loss,label='Mean Loss')
plt.plot(x,epoch_med_loss, label='Median Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()


In [ ]:
# Save Trained Model to Google Drive

save_path = references_path = '/content/drive/MyDrive/Deep Learning Final Project/model/final_model'

torch.save(net,save_path)

# Load Testing Data

In [ ]:
## Only run once ##

# Load in Covers80 dataset and save feature matrix to Drive #

test_set_path = '/content/drive/MyDrive/Deep Learning Final Project/covers80/'
test_store_path = '/content/drive/MyDrive/Deep Learning Final Project/test_data/test_refs'


save_feature_matrix(test_set_path, test_store_refs_path)



print('Load Completed')


In [ ]:
# Run every time notebook refresehes #

# Load CQT representations of Covers80 dataset
 
covers_path = '/content/drive/MyDrive/Deep Learning Final Project/test_data/test_refscovers'
references_path = '/content/drive/MyDrive/Deep Learning Final Project/test_data/test_refsreferences'

infile = open(covers_path,'rb')
test_cover_songs = pickle.load(infile)
infile.close()

infile = open(references_path,'rb')
test_reference_songs = pickle.load(infile)
infile.close()

# Testing the Network

In [ ]:
save_path = references_path = '/content/drive/MyDrive/Deep Learning Final Project/model/final_model'

# Load Trained Network
model = torch.load(save_path)
model.eval()

count = 0

rank_mat = []

# For every reference song, compare to every cover song

for reference_song in test_reference_songs:
  song0 = torch.from_numpy(test_reference_songs[reference_song]).cuda()
  song0 = song0.unsqueeze(0)
  song0 = song0.unsqueeze(0)
  data = []
  for cover_song in test_cover_songs:
    song1 = torch.from_numpy(test_cover_songs[cover_song]).cuda()
    song1 = song1.unsqueeze(0)
    song1 = song1.unsqueeze(0)
    if cover_song.find(reference_song) == -1:
      test_label = 1
    else: 
      test_label = 0
    with torch.no_grad():
      # Input (reference,cover) to the network 
      output = model(torch.abs(song0).float(),torch.abs(song1).float())
      output_label_tuple = (test_label,output)
      data.append(output_label_tuple)
  # Rank reprsentation from lowest to highest of every cover song versus the reference song    
  data.sort(key=lambda pair: pair[1])

  # Find index of true cover song
  rank = [x for x, y in enumerate(data) if y[0] == 0]
  if len(rank)>1:
    rank.sort()
  rank_mat.append(rank[0])



In [ ]:
# Create boxplot of rankings of true cover song for all reference songs

import matplotlib.pyplot as plt

plt.boxplot(rank_mat)
plt.title("Rank of Cover Songs")
plt.show()